In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# dataset_path = "/Users/elliot/dataset/mnist"
dataset_path = '/home/elliot/data/pytorch/mnist'


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [2]:
model = Net()

In [3]:
model

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [9]:
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        print(m.weight.data)

tensor([[[[ 1.7727e-01,  3.3815e-02, -1.7535e-01,  1.1776e-01, -1.7912e-01],
          [-1.7095e-01,  3.0693e-02,  1.7659e-01,  1.4220e-01, -9.1090e-02],
          [-1.9096e-02, -1.2771e-01,  7.1549e-02, -8.3423e-02, -1.6167e-02],
          [-3.5838e-02, -2.1796e-02, -1.8951e-01,  8.3628e-02,  1.8649e-01],
          [-1.6090e-03,  1.4975e-01,  5.0063e-02, -6.6054e-02,  1.8455e-01]]],


        [[[-1.6922e-01,  1.1864e-01,  1.0238e-01, -1.2225e-01, -8.3001e-02],
          [-2.5274e-03,  6.4428e-04, -3.7204e-02, -4.1531e-02,  1.9164e-01],
          [ 1.7943e-01,  1.7268e-01,  8.6356e-02, -1.0641e-01,  5.4122e-02],
          [-3.6004e-02,  1.6400e-01,  1.4366e-01,  1.0525e-02,  7.5099e-02],
          [-1.7169e-01,  1.7790e-01, -1.9665e-01, -1.3416e-01, -2.6474e-03]]],


        [[[ 1.9744e-01, -1.7434e-01, -1.6414e-02,  9.5021e-02, -1.4731e-01],
          [-9.2433e-02,  3.7032e-03,  7.5376e-02,  4.7721e-02,  5.1283e-02],
          [ 1.0185e-02, -1.5183e-01, -8.9655e-02, -1.9975e-01, -2.42